In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_income_statement")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id              5759 non-null   int64              
 1   ticker          5759 non-null   object             
 2   stac_yymm       5759 non-null   object             
 3   sale_account    5759 non-null   object             
 4   sale_cost       5759 non-null   object             
 5   sale_totl_prfi  5759 non-null   object             
 6   depr_cost       5759 non-null   object             
 7   sell_mang       5759 non-null   object             
 8   bsop_prti       5759 non-null   object             
 9   bsop_non_ernn   5759 non-null   object             
 10  bsop_non_expn   5759 non-null   object             
 11  op_prfi         5759 non-null   object             
 12  spec_prfi       5759 non-null   object             
 13  spec_loss       5759 non-null   o

,id,ticker,stac_yymm,sale_account,sale_cost,sale_totl_prfi,depr_cost,sell_mang,bsop_prti,bsop_non_ernn,bsop_non_expn,op_prfi,spec_prfi,spec_loss,thtr_ntin,updated_at
0,1,005930,202506,1537068.00,1000795.00,536273,99.99,99.99,113613.00,99.99,99.99,149077.00,99.99,99.99,133393.00,2025-09-16 13:00:10.396469+00:00
1,2,005930,202503,791405.00,510099.00,281306,99.99,99.99,66853.00,99.99,99.99,91516.00,99.99,99.99,82229.00,2025-09-16 13:00:10.396469+00:00
2,3,005930,202412,3008709.00,1865623.00,1143086,99.99,99.99,327260.00,99.99,99.99,375297.00,99.99,99.99,344514.00,2025-09-16 13:00:10.396469+00:00
3,4,005930,202409,2250826.00,1392934.00,857892,99.99,99.99,262333.00,99.99,99.99,296225.00,99.99,99.99,266970.00,2025-09-16 13:00:10.396469+00:00
4,5,005930,202406,1459839.00,901984.00,557855,99.99,99.99,170499.00,99.99,99.99,193021.00,99.99,99.99,165961.00,2025-09-16 13:00:10.396469+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5754,5755,002710,201903,1057.00,972.00,85,99.99,99.99,39.00,99.99,99.99,26.00,99.99,99.99,22.00,2025-09-16 13:00:10.396469+00:00
5755,5756,002710,201812,4132.00,3779.00,353,99.99,99.99,171.00,99.99,99.99,154.00,99.99,99.99,98.00,2025-09-16 13:00:10.396469+00:00
5756,5757,002710,201809,3027.00,2798.00,229,99.99,99.99,103.00,99.99,99.99,90.00,99.99,99.99,60.00,2025-09-16 13:00:10.396469+00:00
5757,5758,002710,201806,1977.00,1831.00,146,99.99,99.99,63.00,99.99,99.99,47.00,99.99,99.99,29.00,2025-09-16 13:00:10.396469+00:00


In [3]:
final_cols = column_index[1:]

In [4]:
for col in final_cols:
  df[col] .replace(r'^\s*$', np.nan, regex=True, inplace=True)
  df[col].replace(['0', 0], np.nan, inplace=True)

missing_value_counts = df[final_cols].isnull().sum()

missing_value_counts

ticker            0
stac_yymm         0
sale_account      0
sale_cost         0
sale_totl_prfi    4
depr_cost         0
sell_mang         0
bsop_prti         0
bsop_non_ernn     0
bsop_non_expn     0
op_prfi           0
spec_prfi         0
spec_loss         0
thtr_ntin         0
updated_at        0
dtype: int64

In [5]:
missing_cols = missing_value_counts[missing_value_counts >= 100].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : []
[원본 컬럼 : 15] - [결측 컬럼 : 0] = [후보 컬럼 : 15]


In [6]:
nunique_counts = df[final_cols].nunique()

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['depr_cost', 'sell_mang', 'bsop_non_ernn', 'bsop_non_expn', 'spec_prfi', 'spec_loss', 'updated_at']
[원본 컬럼 : 15] - [결측 컬럼 : 0] - [유니크 컬럼 : 7] = [후보 컬럼 : 8]


In [7]:
df[final_cols]

,ticker,stac_yymm,sale_account,sale_cost,sale_totl_prfi,bsop_prti,op_prfi,thtr_ntin
0,005930,202506,1537068.00,1000795.00,536273,113613.00,149077.00,133393.00
1,005930,202503,791405.00,510099.00,281306,66853.00,91516.00,82229.00
2,005930,202412,3008709.00,1865623.00,1143086,327260.00,375297.00,344514.00
3,005930,202409,2250826.00,1392934.00,857892,262333.00,296225.00,266970.00
4,005930,202406,1459839.00,901984.00,557855,170499.00,193021.00,165961.00
...,...,...,...,...,...,...,...,...
5754,002710,201903,1057.00,972.00,85,39.00,26.00,22.00
5755,002710,201812,4132.00,3779.00,353,171.00,154.00,98.00
5756,002710,201809,3027.00,2798.00,229,103.00,90.00,60.00
5757,002710,201806,1977.00,1831.00,146,63.00,47.00,29.00


In [8]:
print(final_cols)

['ticker', 'stac_yymm', 'sale_account', 'sale_cost', 'sale_totl_prfi', 'bsop_prti', 'op_prfi', 'thtr_ntin']


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | **필수** | 어떤 기업의 실적인지를 특정하는 핵심 식별자입니다. 모든 데이터를 연결하기 위해 반드시 필요합니다. |
| `stac_yymm` | 결산년월 | **필수** | 데이터가 어느 시점의 실적인지를 나타내는 기준점으로, 시계열 분석 및 특정 시점의 뉴스 이벤트와 결합할 때 필수적입니다. |
| `sale_account` | 매출액 | **필수** | 기업의 외형적 규모와 성장을 보여주는 가장 기본적인 지표입니다. 어닝 서프라이즈/쇼크를 판단하는 첫 번째 기준이 됩니다. |
| `sale_cost` | 매출원가 | **중** | 매출에 직접적으로 대응하는 비용으로, 매출총이익을 계산하는 데 필요합니다. 원가율 변화는 기업의 수익성 변화를 나타냅니다. |
| `sale_totl_prfi`| 매출총이익 | **상** | 매출액에서 매출원가를 뺀 이익으로, 기업의 핵심적인 상품/서비스 자체의 수익성을 보여주는 중요한 지표입니다. |
| `bsop_prti` | 영업이익 | **필수** | 매출총이익에서 판관비를 제외한 이익으로, 기업의 주된 영업활동에서 발생한 성과를 나타냅니다. 시장에서 가장 주목하는 이익 지표 중 하나입니다. |
| `op_prfi` | 경상이익 | **중** | 영업이익에 영업 외 손익을 더한 값입니다. 기업의 전반적인 이익 창출 능력을 보여주지만, 일회성 요인이 포함될 수 있어 영업이익보다 중요도는 낮습니다. |
| `thtr_ntin` | 당기순이익 | **상** | 모든 비용과 세금을 제외한 최종적인 이익으로, EPS(주당순이익) 계산의 기반이 되는 핵심 데이터입니다. |

In [9]:
final_cols = [
  'ticker',
  'stac_yymm',
  'sale_account',
  'sale_cost',
  'sale_totl_prfi',
  'bsop_prti',
  'op_prfi',
  'thtr_ntin'
]

In [10]:
df[final_cols]

,ticker,stac_yymm,sale_account,sale_cost,sale_totl_prfi,bsop_prti,op_prfi,thtr_ntin
0,005930,202506,1537068.00,1000795.00,536273,113613.00,149077.00,133393.00
1,005930,202503,791405.00,510099.00,281306,66853.00,91516.00,82229.00
2,005930,202412,3008709.00,1865623.00,1143086,327260.00,375297.00,344514.00
3,005930,202409,2250826.00,1392934.00,857892,262333.00,296225.00,266970.00
4,005930,202406,1459839.00,901984.00,557855,170499.00,193021.00,165961.00
...,...,...,...,...,...,...,...,...
5754,002710,201903,1057.00,972.00,85,39.00,26.00,22.00
5755,002710,201812,4132.00,3779.00,353,171.00,154.00,98.00
5756,002710,201809,3027.00,2798.00,229,103.00,90.00,60.00
5757,002710,201806,1977.00,1831.00,146,63.00,47.00,29.00


In [11]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m', '%Y%m%d']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ticker          5759 non-null   category      
 1   stac_yymm       5759 non-null   datetime64[ns]
 2   sale_account    5759 non-null   object        
 3   sale_cost       5759 non-null   object        
 4   sale_totl_prfi  5755 non-null   object        
 5   bsop_prti       5759 non-null   object        
 6   op_prfi         5759 non-null   object        
 7   thtr_ntin       5759 non-null   object        
dtypes: category(1), datetime64[ns](1), object(6)
memory usage: 336.0+ KB


In [13]:
cols_to_check = ['sale_account', 'sale_cost', 'sale_totl_prfi', 'bsop_prti', 'op_prfi', 'thtr_ntin']

# 1. 모든 컬럼을 숫자 타입으로 먼저 변환
df[cols_to_check] = df[cols_to_check].apply(pd.to_numeric, errors='coerce')


# 2. 각 컬럼별로 정수 형태 데이터의 비율을 계산하는 함수를 적용
def get_integer_like_ratio(series: pd.Series) -> float:
  """
  Series의 유효한 값(NaN 제외) 중, 정수 형태인 값의 비율을 계산합니다.
  """
  # NaN 값을 제외한 데이터만 추출
  series_without_na = series.dropna()
  
  # 유효한 데이터가 없는 경우, 1.0 (100%) 반환
  if series_without_na.empty:
    return 1.0
    
  # (정수 형태인 값의 개수) / (전체 유효 데이터의 개수)
  integer_like_count = (series_without_na % 1 == 0).sum()
  total_valid_count = len(series_without_na)
  
  return integer_like_count / total_valid_count

# --- 결과 확인 ---
integer_like_ratio_result = df[cols_to_check].apply(get_integer_like_ratio)
print(integer_like_ratio_result)

sale_account      1.0
sale_cost         1.0
sale_totl_prfi    1.0
bsop_prti         1.0
op_prfi           1.0
thtr_ntin         1.0
dtype: float64


| 컬럼명 | 한글컬럼명 | 현재 타입 | 적정 타입 | 이유 |
| :--- | :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | `category` | `category` | 반복적으로 나타나는 종목 코드는 메모리 효율을 위해 `category` 타입으로 관리하는 것이 가장 좋습니다. |
| `stac_yymm` | 결산년월 | `datetime64[ns]` | `datetime64[ns]` | `YYYYMM` 형식의 날짜 데이터는 `pd.to_datetime` 사용 시 `format='%Y%m'`을 지정하여 `datetime`으로 변환해야 합니다. |
| `sale_account` | 매출액 | `object` | `Int64` | 금액 데이터이며 소수점을 포함하고 있으므로, 부동소수점 타입인 `Int64`로 변환해야 정확한 연산이 가능합니다. |
| `sale_cost` | 매출원가 | `object` | `Int64` | 다른 재무 수치와 마찬가지로 소수점을 포함하는 금액 데이터이므로 `Int64` 타입이 적합합니다. |
| `sale_totl_prfi`| 매출총이익 | `object` | `Int64` | 데이터 샘플에서는 정수로 나타나므로, 결측치를 안전하게 처리할 수 있는 Nullable Integer `Int64` 타입이 효율적입니다. (소수점 존재 시 `Int64`) |
| `bsop_prti` | 영업이익 | `object` | `Int64` | 소수점을 포함하는 금액 데이터이므로 `Int64` 타입으로 변환해야 합니다. |
| `op_prfi` | 경상이익 | `object` | `Int64` | 소수점을 포함하는 금액 데이터이므로 `Int64` 타입으로 변환해야 합니다. |
| `thtr_ntin` | 당기순이익 | `object` | `Int64` | 소수점을 포함하는 금액 데이터이므로 `Int64` 타입으로 변환해야 합니다. |

In [14]:
dtype_map = {
  'Int64': ['sale_account', 'sale_cost', 'sale_totl_prfi', 'bsop_prti', 'op_prfi', 'thtr_ntin']
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ticker          5759 non-null   category      
 1   stac_yymm       5759 non-null   datetime64[ns]
 2   sale_account    5759 non-null   Int64         
 3   sale_cost       5759 non-null   Int64         
 4   sale_totl_prfi  5755 non-null   Int64         
 5   bsop_prti       5759 non-null   Int64         
 6   op_prfi         5759 non-null   Int64         
 7   thtr_ntin       5759 non-null   Int64         
dtypes: Int64(6), category(1), datetime64[ns](1)
memory usage: 369.7 KB
